# Filtering Experiments


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path
import polars as pl
import pandas as pd
import plotly.graph_objects as go
import utm

repo_root = Path(os.getcwd()).parent

while not (repo_root / ".git").exists():
    repo_root = repo_root.parentz

sys.path.append(str(repo_root))


## IO


In [3]:
# set the pandas plotting backend to plotly
pd.options.plotting.backend = "plotly"


df = pl.scan_parquet(
    "/Users/max/Library/CloudStorage/Box-Box/Radar-Data/new_format/1678658102006.parquet"
).collect()


## Filter the Data


In [4]:
from src.filtering import Filtering

## Read in the Data

# create the file paths
network_outline_file = repo_root / "geo_data" / "network_outline.geojson"
radar_locations_file = repo_root / "geo_data" / "radar_origins.json"

f = Filtering(
    radar_location_path=radar_locations_file,
    network_boundary_path=network_outline_file,
)


In [5]:
df = (
    df.pipe(f.create_object_id)
    # resample to 10 Hz
    .pipe(f.resample, 100)
    # remove objects that travel for very little time
    .pipe(f.filter_short_trajectories, minimum_distance_m=100, minimum_duration_s=5)
    # clip trajectories to not include the constant speed at the end
    .pipe(f.clip_trajectory_end)
    # rotate the heading measurements to world coordinates
    .pipe(f.rotate_heading)
    # smooth the values during stop events. This is allowed because there is no
    .pipe(f.fix_stop_param_walk)
    # update the centroid coordinates to the actual center of the object
    .pipe(
        f.correct_center,
        x_col="utm_x",
        y_col="utm_y",
    )
    # convert the h3 integer to a string
    .pipe(f.int_h3_2_str)
    # filter out objects that are not in the network
    .pipe(f.filter_network_boundaries)
)


function: resample took: 0.2218949794769287 seconds
function: filter_short_trajectories took: 0.06255888938903809 seconds
function: clip_trajectory_end took: 0.09811997413635254 seconds
function: rotate_heading took: 0.7228538990020752 seconds
function: fix_stop_param_walk took: 0.05876493453979492 seconds
function: correct_center took: 0.004044055938720703 seconds
function: int_h3_2_str took: 0.12333321571350098 seconds
function: filter_network_boundaries took: 0.02138495445251465 seconds


## Kalman Filtering Playground

https://paperswithcode.com/task/trajectory-prediction#:~:text=Trajectory%20Prediction%20is%20the%20problem,rickshaws%2C%20and%20animals%2C%20etc.


In [6]:
_id = "40379~10.160.7.136~2023-03-12"

veh_df = (
    df.filter(pl.col("object_id").is_in([_id]))
    .with_columns(
        [
            (
                (pl.col("epoch_time") - pl.col("epoch_time").min()).cast(pl.Int32) / 1e3
            ).alias("time")
        ]
    )
    .to_pandas()
)


In [281]:
import numpy as np

veh_df = veh_df[
    [
        "epoch_time",
        "utm_x",
        "utm_y",
        "direction",
        "f32_velocityInDir_mps",
        "f32_length_m",
    ]
].copy()

# get the x & y velocity
veh_df["x_vel"] = veh_df["f32_velocityInDir_mps"] * np.cos(veh_df["direction"])
veh_df["y_vel"] = veh_df["f32_velocityInDir_mps"] * np.sin(veh_df["direction"])

veh_df


,epoch_time,utm_x,utm_y,direction,f32_velocityInDir_mps,f32_length_m,x_vel,y_vel
0,2023-03-12 21:57:40.400,442735.620171,3.677566e+06,4.647860,6.017757,4.743290,-0.388052,-6.005233
1,2023-03-12 21:57:40.500,442735.604912,3.677565e+06,4.652147,6.290564,4.743290,-0.378724,-6.279153
2,2023-03-12 21:57:40.600,442735.768707,3.677564e+06,4.679367,6.123858,4.743290,-0.202188,-6.120520
3,2023-03-12 21:57:40.700,442735.638265,3.677564e+06,4.665418,6.059743,4.743290,-0.284524,-6.053060
4,2023-03-12 21:57:40.800,442735.531827,3.677563e+06,4.655951,5.888338,4.743290,-0.332150,-5.878963
...,...,...,...,...,...,...,...,...
883,2023-03-12 21:59:08.800,442757.465933,3.677494e+06,5.434317,6.910506,5.581761,4.566692,-5.186562
884,2023-03-12 21:59:08.900,442758.155846,3.677493e+06,5.480903,6.970448,5.581761,4.844932,-5.011366
885,2023-03-12 21:59:09.000,442758.901875,3.677493e+06,5.515099,7.109049,5.581761,5.113122,-4.939086
886,2023-03-12 21:59:09.100,442759.589008,3.677493e+06,5.549002,7.200565,5.581761,5.345544,-4.824241


### Stone Soup

In [282]:
from datetime import datetime, timedelta

from stonesoup.models.transition.linear import (
    ConstantAcceleration,
    ConstantVelocity,
    CombinedLinearGaussianTransitionModel,
)

from stonesoup.models.transition.nonlinear import ConstantTurn

from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.predictor.kalman import UnscentedKalmanPredictor
from stonesoup.updater.kalman import UnscentedKalmanUpdater
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.types.detection import Detection

from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.types.state import GaussianState


In [283]:
transition_model = CombinedLinearGaussianTransitionModel(
    [
        ConstantAcceleration(1),
        ConstantAcceleration(1),
    ]
)
transition_model.matrix(time_interval=timedelta(seconds=0.1))


measurement_model = LinearGaussian(
    ndim_state=6,  # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 1, 3, 4),  # Mapping measurement vector index to state index
    # noise_covar=np.diag([1 / 20, 1/ 160, 1 / 20, 1/ 160]),  # Covariance matrix for Gaussian PDF
    noise_covar=np.diag(
        [5 for _ in range(4)]
    ),  # Covariance matrix for Gaussian PDF
)

# This maps the state vector to the measurement vector
measurement_model.matrix()


predictor = KalmanPredictor(transition_model)
updater = KalmanUpdater(measurement_model)

# create measurements
measurements = [
    Detection(
        np.array([[row.utm_x], [row.x_vel], [row.utm_y], [row.y_vel]]),
        timestamp=row.epoch_time,
        measurement_model=measurement_model,
    )
    for i, row in veh_df.iterrows()
]

prior = GaussianState(
    [
        measurements[0].state_vector[0, 0],
        measurements[0].state_vector[1, 0],
        1,
        measurements[0].state_vector[2, 0],
        measurements[0].state_vector[3, 0],
        1,
    ],
    np.diag([100 for _ in range(6)]),
    timestamp=measurements[0].timestamp,
)


In [284]:
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track

track = Track()
for measurement in measurements:
    prediction = predictor.predict(prior, timestamp=measurement.timestamp)
    hypothesis = SingleHypothesis(
        prediction, measurement
    )  # Group a prediction and measurement
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

# extend the predictions by 5 seconds
for _ in range(30):
    prediction = predictor.predict(
        track[-1], timestamp=track[-1].timestamp + timedelta(seconds=0.1)
    )
    track.append(prediction)


In [285]:
# plot the learned velocity vs time
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=veh_df.epoch_time,
        y=veh_df.f32_velocityInDir_mps,
        mode="lines",
        opacity=1,
        line_color="black",
        name="measured",
        showlegend=True,
        # hover the velocity

    )
)

# # plot velocity on the x axis
# fig.add_trace(
#     go.Scatter(
#         x=[x.timestamp for x in track],
#         y=[x.state_vector[0, 0] for x in track],
#         mode="lines",
#         opacity=1,
#         line_color="red",


fig.add_trace(
    go.Scatter(
        x=[x.timestamp for x in track],
        y=[
            np.sqrt(x.state_vector[1, 0] ** 2 + x.state_vector[4, 0] ** 2)
            for x in track
        ],
        # y = [x.state_vector[0, 0] for x in track],
        mode="lines",
        opacity=1,
        line_color="red",
        name="kalman",
        showlegend=True,
    )
)


In [286]:
from stonesoup.plotter import Plotterly

plotter = Plotterly()
plotter.plot_measurements(measurements, [0, 3])

plotter.plot_tracks(track, [0, 3], uncertainty=True)
plotter.fig


### Tring with CTRA

From https://github.com/balzer82/Kalman/blob/master/Extended-Kalman-Filter-CTRA.ipynb

### Trying with my Implementation


In [13]:
from src.kalman import KalmanConstantAcceleration, CTRAModel
from src.kalman import (
    KalmanConstantAcceleration,
    KalmanConstantAccelerationFadingMemory,
)
from filterpy.kalman import KalmanFilter
from plotly.subplots import make_subplots

from filterpy.kalman import UnscentedKalmanFilter
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import MerweScaledSigmaPoints


In [278]:
# _id = "74507~10.160.7.146~2023-03-12"
# veh_df = (
#     df.filter(pl.col("object_id").is_in(df['object_id'].sample(1).to_list()))
#     # df.filter(pl.col("object_id").is_in([_id]))
#     .with_columns(
#         [
#             (
#                 (pl.col("epoch_time") - pl.col("epoch_time").min()).cast(pl.Int32) / 1e3
#             ).alias("time")
#         ]
#     )
#     .to_pandas()
# )


measurements = veh_df[["utm_x", "utm_y", "direction", "f32_velocityInDir_mps"]].values

model = CTRAModel(0.1)

kf = UnscentedKalmanFilter(
    dim_x=model.dim_x,
    dim_z=model.dim_z,
    dt=model.dt,
    fx=model.F,
    hx=model.H,
    points=MerweScaledSigmaPoints(model.dim_x, alpha=0.01, beta=2, kappa=0, ),
)

kf.x = np.r_[measurements[0], 0.1, 0.0000]
kf.P = np.diag([0.1 for _ in range(model.dim_x)]) * 1
kf.R = model.R 
kf.Q = model.Q(kf.x)


track = []
for z in measurements[1:]:
    kf.predict(
        UT=model.unscented_transform,
    )
    kf.update(z)
    track.append(kf.x)


LinAlgError: 2-th leading minor of the array is not positive definite

In [279]:

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Position", "Velocity"),
    specs=[[{"type": "scatter"}, {"type": "scatter"}]],
)

# add the x/y position
fig.add_trace(
    go.Scatter(
        x=veh_df.utm_x,
        y=veh_df.utm_y,
        mode="lines",
        opacity=1,
        line_color="black",
        name="measured",
    ),
    row=1,
    col=1,
)

# add the x/y position
fig.add_trace(
    go.Scatter(
        x=[t[0] for t in track],
        y=[t[1] for t in track],
        mode="lines",
        opacity=1,
        line_color="red",
        name="kalman",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=veh_df.epoch_time,
        y=veh_df.f32_velocityInDir_mps,
        mode="lines",
        opacity=1,
        line_color="black",
        name="measured",
        showlegend=True,
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(
        x=veh_df.epoch_time.min() + (timedelta(seconds=0.1) * np.arange(len(track))),
        y=[t[3] for t in track],
        mode="lines",
        opacity=1,
        line_color="red",
        name="kalman",
        showlegend=True,
    ),
    row=1,
    col=2,
)


# fig.add_trace(
#     go.Scatter(
#         x=veh_df.epoch_time.min() + (timedelta(seconds=0.1) * np.arange(len(track))),
#         y=[np.sqrt(t[1] ** 2 + t[4] ** 2) for t in track_adapt],
#         mode="lines",
#         opacity=1,
#         line_color="blue",
#         name="kalman - adaptive",
#         showlegend=True,
#     ),
#     row=1,
#     col=2,
# )


fig.show()

In [280]:
# plot x vs velocity

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=veh_df.utm_x,
        y=veh_df.f32_velocityInDir_mps,
        mode="lines",
        opacity=1,
        line_color="black",
        name="measured",
        showlegend=True,
    )
)

In [277]:
# plot the heading vs time

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=veh_df.epoch_time,
        y=veh_df.direction,
        mode="lines",
        opacity=1,
        line_color="black",
        name="measured",
        showlegend=True,
    )
)

fig.add_trace(
    go.Scatter(
        x=veh_df.epoch_time.min() + (timedelta(seconds=0.1) * np.arange(len(track))),
        y=[t[2] for t in track],
        mode="lines",
        opacity=1,
        line_color="red",
        name="kalman",
        showlegend=True,
    )
)


## Make it Into A Polars Function to Do Positional and Velocity Smoothing


- This should be applied to each vehicle individually


In [15]:
def kalman_filter(df: pl.DataFrame) -> pl.DataFrame:
    measurements = df[["epoch_time", "utm_x", "x_vel", "utm_y", "y_vel"]].to_numpy()

    # create the kalman filter
    kf = KalmanConstantAccelerationFadingMemory(dt=0.1, x0=measurements[0, 1:], alpha=1.02)

    # run the kalman filter
    measurements[:, 1:] = kf.batch_update(measurements[:, 1:])[:, (0, 1, 3, 4)]
    # smooth the results
    # measurements[:, 1:] = np.array(kf.rts_smooth()[0])[:, (0, 1, 3, 4)]

    new_measurements = [
        [measurements[-1, 0] + (t * 100), *kf.predict()[[0, 1, 3, 4]]]
        for t in range(30)
    ]
    measurements = np.vstack((measurements, np.array(new_measurements)))


    return pl.DataFrame(
        data=measurements,
        schema=["epoch_time", "utm_x", "x_vel", "utm_y", "y_vel"],
    ).with_columns(
        [
            pl.lit(df[c].take(0)).alias(c)
            for c in df.columns
            if c not in ["epoch_time", "utm_x", "x_vel", "utm_y", "y_vel"]
        ]
    )

In [16]:
return_df = (
    df.with_columns(
        [
            (pl.col("f32_velocityInDir_mps") * pl.col("direction").cos()).alias(
                "x_vel"
            ),
            (pl.col("f32_velocityInDir_mps") * pl.col("direction").sin()).alias(
                "y_vel"
            ),
        ]
    )
    .groupby("object_id")
    .apply(kalman_filter)
)


In [17]:
return_df = return_df.sort(['object_id', 'epoch_time']).with_columns(
    [
        ((pl.col("x_vel") ** 2 + pl.col("y_vel") ** 2) ** 0.5).alias("filtered_speed"),
        # calculate the direction
        # pl.col("utm_x").atan2(pl.col("y_vel")).alias("filtered_direction"),
        pl.col("utm_x").diff().backward_fill().over('object_id').alias("x_diff"),
        pl.col("utm_y").diff().backward_fill().over('object_id').alias("y_diff"),
        # cast the epoch time to a datetime
        (pl.col("epoch_time") * 1e3).cast(pl.Datetime).alias("epoch_time"),
    ]
).with_columns([
    (pl.struct([pl.col("x_diff"), pl.col("y_diff")]).apply(
        lambda x: np.arctan2(x['y_diff'], x['x_diff'])
    )).alias("filtered_direction")
]).drop(["x_diff", "y_diff"])

## Kalman Filter Trajectories


In [18]:
## Get a random id
id_ = return_df.select("object_id").unique().sample(1)['object_id'].to_numpy()[0]

In [19]:
id_ 


'46342~10.160.7.137~2023-03-12'

In [20]:
veh_df = df.filter(pl.col("object_id") == id_).sort("epoch_time").to_pandas()
kalman_vehicle_df = return_df.filter(pl.col("object_id") == id_).sort("epoch_time").to_pandas()

In [21]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Position", "Speed"),
    specs=[[{"type": "scatter"}, {"type": "scatter"}]],
)

fig.add_trace(
    go.Scatter(
        x=veh_df.utm_x,
        y=veh_df.utm_y,
        mode="markers",
        opacity=1,
        marker_color="black",
        name="measured",
        showlegend=True,
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=kalman_vehicle_df.utm_x,
        y=kalman_vehicle_df.utm_y,
        mode="lines",
        opacity=1,
        line_color="red",
        name="measured",
        showlegend=True,
    ),
    row=1,
    col=1,
)



fig.add_trace(
    go.Scatter(
        x=veh_df.epoch_time,
        y=veh_df.f32_velocityInDir_mps,
        mode="markers",
        opacity=1,
        marker_color="black",
        name="measured",
        showlegend=True,
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(
        x=kalman_vehicle_df.epoch_time,
        y=kalman_vehicle_df.filtered_speed,
        mode="lines",
        opacity=1,
        line_color="red",
        name="measured",
        showlegend=True,
    ),
    row=1,
    col=2,
)

fig.show()
